> 我们可以把给llm的提示词预选做成模板,实现提示词的复用,版本管理,动态变化等

In [1]:
import os
os.environ["OPENAI_KEY"] = "sk-bQGeRZevfQI95TPb1b1a3e80011c4f368150270f687dF163"
os.environ["OPENAI_API_BASE"] = "https://api.aihubmix.com/v1"

# PromptTemplate
普通的字符串模板

In [30]:
#字符模板
from langchain.prompts import PromptTemplate

# 定义模板
prompt = PromptTemplate.from_template("你是一个{name},帮我起1个具有{county}特色的{sex}名字")
# 使用模板生成提示词
prompt.format(name="算命大师", county="法国", sex="女孩")

'你是一个算命大师,帮我起1个具有法国特色的女孩名字'

# ChatPromptTemplate
对话模型模板
 

In [2]:
# 对话模板具有结构，对应-chat models
from langchain.prompts import ChatPromptTemplate

# system,human,ai,assistant是系统的默认角色
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个起名大师. 你的名字叫{name}."),
        ("human", "你好{name},你感觉如何？"),
        ("ai", "你好！我状态非常好!"),
        ("human", "你叫什么名字呢?"),
        ("ai", "你好！我叫{name}"),
        ("human", "{user_input}"),
    ]
)

chat_template.format_messages(name="陈大师", user_input="你的父亲是谁呢?")

# 我们可以解析后的chat_template传递给大模型
# 最后一句话就是我们的问题,之前的对话就是提供给大模型的上下文,上下文越多回答越准确

[SystemMessage(content='你是一个起名大师. 你的名字叫陈大师.'),
 HumanMessage(content='你好陈大师,你感觉如何？'),
 AIMessage(content='你好！我状态非常好!'),
 HumanMessage(content='你叫什么名字呢?'),
 AIMessage(content='你好！我叫陈大师'),
 HumanMessage(content='你的父亲是谁呢?')]

可以使用langchain.schema中提供的方法替代上述写法

In [32]:

from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

# 直接创建消息
sy = SystemMessage(
    content="你是一个起名大师",
    additional_kwargs={"大师姓名": "陈瞎子"}
)

hu = HumanMessage(
    content="请问大师叫什么?"
)
ai = AIMessage(
    content="我叫陈瞎子"
)
[sy, hu, ai]


[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师姓名': '陈瞎子'}),
 HumanMessage(content='请问大师叫什么?'),
 AIMessage(content='我叫陈瞎子')]

langchain帮我们把这几类角色都初始化好了,可以直接使用

In [33]:
# langchain帮我们把这几类角色都初始化好了,可以直接使用
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
# 支持自定义角色
from langchain.prompts import ChatMessagePromptTemplate

prompt = "愿{subject}与你同在！"

# ChatMessagePromptTemplate模板的作用是支持我们自定义角色
chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者", template=prompt)
#chat_message_prompt = AIMessagePromptTemplate.from_template(template=prompt) 默认就是AI角色

chat_message_prompt.format(subject="原力")

ChatMessage(content='愿原力与你同在！', role='天行者')

# 自定义模板
场景: 函数大师：根据函数名称，查找函数代码，并给出中文的代码说明

In [34]:
from langchain.prompts import StringPromptTemplate
# py内置类,方便我们通过方法名获取方法源代码
import inspect


# 定义一个简单的函数作为示例效果
def add(i, j):
    return i + j


PROMPT = """
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: {function_name}
源代码: 
    {source_code}
代码解释:
"""


#自定义的模板class
# **kwargs :接收任意数量的关键字参数
# def example_function(**kwargs):
#     name = kwargs["name"]
#     for key, value in kwargs.items():
#         print(key, value)
# 
# # 调用
# example_function(name='John', age=25, city='New York')

# 这种写法表示继承
class CustmPrompt(StringPromptTemplate):  

    def format(self, **kwargs) -> str:
        # 从参数获取
        function_name = kwargs["function_name"]
        # 生成提示词模板
        prompt = PROMPT.format(
            function_name=function_name.__name__,  # __name__ 获取函数名的默认属性
            source_code=inspect.getsource(function_name)
        )
        return prompt


a = CustmPrompt(input_variables=["function_name"])
pm = a.format(function_name=add)

print(pm)


你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: add
源代码: 
    def add(i, j):
    return i + j

代码解释:



In [35]:
#和LLM连接起来
from langchain.llms import OpenAI
import os

llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE")
)
msg = llm.invoke(pm)
print(msg)

函数名称: add
源代码: 
    def add(i, j):
    return i + j

代码解释:
这是一个名为add的函数，它接受两个参数i和j，并返回它们的和。


# 使用jinji2与f-string来实现提示词模板格式化
这2都是 python支持的字符串处理工具

In [36]:
##f-string是python内置的一种模板引擎
from langchain.prompts import PromptTemplate

fstring_template = """
给我讲一个关于{name}的{what}故事
"""

prompt = PromptTemplate.from_template(fstring_template)
print(prompt.format(name="翠花", what="悲伤"))


给我讲一个关于翠花的悲伤故事



In [37]:
# ! pip install jinja2

In [38]:
##Jinja2是一个灵活、高效的Python模板引擎，可以方便地生成各种标记格式的文档。
from langchain.prompts import PromptTemplate
## 使用2个括号来标注关键词
jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(name="狗剩", what="高兴")

'给我讲一个关于狗剩的高兴故事'

# 组合式提示词模板
> 针对一些提示词比较复杂的场景,我们可以把提示词做拆分,然后合并
- Final prompt: 最终返回的提示词模板
- Pipeline prompts：组成提示词管道的模板

* 三层提示词设计

In [44]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Final Prompt由一系列变量构成
full_template = """
{Character}
{behavior}
{prohibit}"""
full_prompt = PromptTemplate.from_template(full_template)

* 定义每一层的提示词模板

In [45]:
# 第一层基本性格设计
Character_template = """你是{person}，你有着{character}."""
Character_prompt = PromptTemplate.from_template(Character_template)

# 第二层行为设计
behavior_template = """你遵从以下的行为:
{behavior_list}"""
behavior_prompt = PromptTemplate.from_template(behavior_template)

# 第三层不允许的行为
prohibit_template = """你不允许有以下行为:
{prohibit_list}"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

- 将三层提示称组合起来

In [46]:
input_prompts = [
    ("Character", Character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

- 打印下组合后模板里的可传递参数

In [47]:
pipeline_prompt.input_variables

['person', 'behavior_list', 'character', 'prohibit_list']

- 调用组合后的提示词模板

In [49]:
pm = pipeline_prompt.format(
    person="埃隆马斯克",
    character="钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星.",
    behavior_list=" 1.你喜欢冒险 \n 2.你非常崇拜爱因斯坦 \n 3.你总是乐于助人，回答用户各种问题.",
    prohibit_list=" 1.你不可以说自己是一个人工智能助手或者机器人 \n 2.当你不知道答案的时候，可以说让我再想想. "
)
print(pm)


你是埃隆马斯克，你有着钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星..
你遵从以下的行为:
 1.你喜欢冒险 
 2.你非常崇拜爱因斯坦 
 3.你总是乐于助人，回答用户各种问题.
你不允许有以下行为:
 1.你不可以说自己是一个人工智能助手或者机器人 
 2.当你不知道答案的时候，可以说让我再想想. 


# 序列化：使用文件来管理提示词模板

- 便于共享
- 便于版本管理
- 便于存储
- 支持常见格式(json/yaml/txt)

## yarn格式支持

In [2]:
from langchain.prompts import load_prompt

In [3]:
#加载yaml格式的prompt模版
prompt = load_prompt("testfile/simple_prompt.yaml")
print(prompt.format(name="小黑",what="恐怖的"))

给我讲一个关于小黑的恐怖的故事


## json格式支持

In [4]:
#加载json格式的prompt模版
prompt = load_prompt("testfile/simple_prompt.json")
print(prompt.format(name="小红",what="搞笑的"))

给我讲一个关于小红的搞笑的故事
